In [230]:
# Add caption and unit in the, and axis x and y; Prove guesses

# Green Taxi Trips!
This report shows my analysis on Green Taxi trip data. The data is collected by the New York City Taxi and Limousine commission (TLC) about “Green” Taxis. It contains trip information of Green Taxis in September, 2015.

The data includes fields capturing pick-up and drop-off dates/times, pick-up and drop-off locations, trip distances, itemized fares, rate types, payment types, and driver-reported passenger counts. Please refer the following file for description of each field. http://www.nyc.gov/html/tlc/downloads/pdf/data_dictionary_trip_records_green.pdf

It consists of six parts:

* Data overview

* Distribution of 'Trip_distance'

* Analysis of 'Trip_distance' grouped by hour of day; Identification of trips that originate or terminate at one of the NYC area airports.

* Predictive modeling for tip percentage

* Distribution of average speed 

* Appendix: python code

Some background information of Green Taxis:

Green Taxis (as opposed to yellow ones) are taxis in New York City that are only allowed to pick up passengers (street hails or calls) in outer boroughs (excluding John F. Kennedy International Airport and LaGuardia Airport unless arranged in advance) and in Manhattan above East 96th and West 110th Streets. 

Before Green Taxis exist, an analysis conducted by TLC showed that about 95% of yellow taxi pick-ups occurred in Manhattan below 96th Street and at JFK and LaGuardia (LGA) airports. This resulted in significantly lower access to legal taxi rides for people in outer boroughs. As a result, the Five Borough Taxi Plan was started with the Street Hail Livery program to allow "Green Taxis" to pick up street-hail passengers to fill in the gap.[1]



# Q1: Data Overview

The data includes about 1.5 million trips and 21 fields. I grouped fields into four categories based on their characteristics:

* Location related fields: 'Pickup_longitude', 'Pickup_latitude', 'Dropoff_longitude', 'Dropoff_latitude'

* Time related fields: 'lpep_pickup_datetime', 'Lpep_dropoff_datetime'

* Fare related fields: 'RateCodeID', 'Fare_amount', 'Extra', 'MTA_tax', 'Tip_amount', 'Tolls_amount', 'improvement_surcharge', 'Total_amount', 'Payment_type'

* Others: 'VendorID', 'Trip_type ', 'Store_and_fwd_flag', 'Passenger_count', 'Trip_distance'

The 'Ehail_Fee' field is null, so we can drop it. 'Trip_type' has 4 null values. All the other fields don't have null values.

The code is in Appendix A.

# Q2: Distribution of 'Trip_distance'

'Trip_distance' is the elapsed trip distance in miles reported by the taximeter. Analyzing the distribution of Trip_distance can help us understand how long a Green taxi trip can be; a step further, may help us promoting the ride sharing.

About 1.4% of all the trips in the data have zero in the 'Trip_distance' field, but their 'Fare_amount' are sometimes positive. I think those zeros may be noise because they can be missing values if corresponding 'Fare_amount'is positive, or those trips didn't even start. Both cases are not useful for the analysis of distribution of trip distance. So I removed trips with 'Trip_distance' equals zero. We may need to contact TLC to check why zero exis in 'Trip_distance' field.

The histogram and boxplot of ’Trip_distance‘ below show that the distribution is left skewed. I guess it may follow log-normal distribution. I'd try to fit the parameters and estimate the distribution if I have extra time. 

<img src="1.png">

The average 'Trip_distance' is 3.01, the median is 2.00, and the variance is 0.47.

0.2% of all the trips (# 3394) in September have 'Trip_distnace' more than 20 miles, and two trips more than 200 miles. I don't think these are outliers, but are special cases. 

The code is in Appendix B.


# Q3
## Engineer time related features 

Since we're going to use hour information to analyze 'Trip_distance', I first extracted hour and weekday information from time related fields('lpep_pickup_datetime', 'Lpep_dropoff_datetime' ), and derived four new fields ('lpep_pickup_hours', 'lpep_pickup_weekday', 'Lpep_dropoff_hour', 'Lpep_dropoff_weekday').

The following distribution plot shows that most trips happened at around 17:00 - 19:00. At around 5:00 am, there are least trips across the day.

<img src="2.png">

Also, I derived a new field 'Time_spent' which indicates the time spent in each trip by calculating the difference between 'lpep_pickup_datetime' and 'Lpep_dropoff_datetime'. The distribution of ’Time_spent‘ is as follows.

The mean of time spent in trip is 0.34, and the median is 0.17. The following distribution plot shows that most trips last less than 1 hours.

<img src="3.png">

The code is in Appendix C1.



## 'Trip_distance' group by hour

Given the analysis in Q2, I removed data of which the 'Trip_distance' equals zero.

Mean, median, variance and count of 'Trip_distance' grouped by hour of day is as follows:

In [228]:
Distance_by_hour

lpep_pickup_hours Trip_distance                         
                              mean median  count        var
0                  0      3.150602   2.23  66405   8.773136
1                  1      3.054612   2.16  53117   8.337778
2                  2      3.091807   2.19  40588   8.803343
3                  3      3.261706   2.26  31167  21.616505
4                  4      3.584065   2.40  26000  12.500216
5                  5      4.214483   2.98  16379  15.726679
6                  6      4.134494   2.90  22232  15.767508
7                  7      3.330973   2.20  41391  10.703169
8                  8      3.088413   2.00  58202   9.619719
9                  9      3.042416   2.00  61144   9.462932
10                10      2.991911   1.97  56557   9.335695
11                11      2.962246   1.90  55828   9.574829
12                12      2.953734   1.91  56836   9.475451
13                13      2.929103   1.89  56480   9.708689
14                14      2.913916   1.87  65529   9.547061
15                15      2.905159   1.86  72555   9.756772
16                16      2.823620   1.83  77930   9.016572
17                17      2.713986   1.80  86891   8.181349
18                18      2.687323   1.81  96011   7.425463
19                19      2.745756   1.88  95085   7.327070
20                20      2.809233   1.92  89745   7.377837
21                21      3.032618   2.07  85589   9.314610
22                22      3.221524   2.21  83755   9.596564
23                23      3.227005   2.26  78918   9.089499

The line plot below shows that 'Trip_distance' at around 5:00 am is the longest, while that around 17:00 is the shortest. 

<img src="4.png">

My hypothesis about this pattern is that: At 17:00, people usually take off from work and then take taxis to party or home which are usually near the work place, so the trip distance is short. 

People usually don't stay awake at 5:00am. The "Distribution of pickup hour of day" above also shows that the number of trips at 5:00am is the smallest. I guess that, if people take taxi at that time, there may be some emergency or special cases, such as catching a flight. In this way, the trip distance is long.

I'd analyze the location distribution across hour of day to test my hypothesis if I have extra time.

The code is in Appendix C2.

## Identify trips that originate or terminate at one of the NYC area airports.

The distribution plots below show that some trips have zero in Pickup_longitude/Pickup_latitude or Dropoff_longitude/Dropoff_latitude. I removed those trips since zero doesn't make sense.

<img src="5.png"> <img src="6.png">

I calculated the distance between pickup/dropoff location and JFK/LGA based on (longitude, latitude) with the code in this link (https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude).

Four new fields are generated:
* 'Pickup_To_JFK': distance between pickup location and JFK
* 'Drop_To_JFK': distance between dropoff location and JFK
* 'Pickup_To_LGA': distance between pickup location and LGA
* 'Drop_To_LGA': distance between dropoff location and JFK

Since I can only get one (longitude, latitude) to represent LGA or JFK, I assume that if the distance between pickup/dropoff location and any one of JFK or LGA is less than 1 mile, those trips originated or terminated at one of the NYC area airports. 

Five new fields are generated:
* 'Is_Pickup_JFK': that trip originated at JFK
* 'Is_Drop_JFK': that trip terminated at JFK
* 'Is_Pickup_LGA': that trip originated at LGA
* 'Is_Drop_LGA': that trip terminated at LGA
* 'Is_Airport': that trip originated or terminated at one of the NYC area airports.

After removing trips with zero in 'Pickup_longitude'/'Pickup_latitude' or 'Dropoff_longitude'/'Dropoff_latitude', about 42k out of all the 1.5 million trips (about 2.8%) originated or terminated at one of the NYC area airports. 

If I have extra time, I'd like to use different criteria (i.e., within 2 miles around airports are taken as originiate / terminate at airports ), and see the changes to result.

I separate all the trips to two groups: those originated or terminated at airports, and the others. The following table shows the average and variance of multiple fields for each group. 


In [232]:
Airport_trips

Fare_amount            Trip_distance            lpep_pickup_hours  \
                 count       mean          mean        var              mean   
Is_Airport                                                                     
False          1449394  12.157139      2.824291   8.059851         13.570220   
True             41983  25.807917      8.057349  31.263511         12.336732   

                      Time_spent            
                  var       mean       var  
Is_Airport                                  
False       46.734237   0.334695  2.630953  
True        27.775585   0.451201  1.860360

Fare Amount is higher for trips related to airports than those not for airports. This may be because those airport-related trips are usually longer in terms of both trip distance and time spent than others.

Code is in Appendix C3.

# Q4: Analysis of tip percentage

Analysis of tip percentage can help us understand the degree of satisfaction of riders, or the behavior pattern of tipping of riders. 

Some trips have negative Tip_amount. The field description indicates that "Tip amount – This field is automatically populated for credit card tips. Cash tips are not included", which mean that 'Tip_amount' field may only be available when 'Payment_type' equals one; when 'Payment_type' is 'Cash', there may be tips but that data is not available.

The boxplot below shows that 'Tip_amount' is around zero or negative when 'Payment_type' is 'No_charge', 'Dispute' or 'Unknown'. For 'Cash', only small amount of trips have positive Tip_amount. So I think we should only keep trips with 'Payment_type' equals one for this analysis. 

<img src="7.png">

The following boxplot shows that some trips have negative or zero 'Total_amount', especially when 'Payment_type' isn't 'Credit_card'. I removed these trips since I'm not able to calculate 'Tip_percent' for them.

 <img src="8.png">

If I have extra time,  I'd like to replace those zero values with the sum of 'Fare_amount', 'Extra', 'MTA_tax', 'improvement_surcharge', 'Tolls_amount' and 'Tip_amount' to make the amount more accurate. 

Code is in Appendix D1.

## Predictive modeling - Derive 'Tip_percent'

Given the analysis above, I only kept trips with 'Payment_type' equals one (Credit_card). 323 records have non positive values for 'Total_amount', and I removed those records.

Then, I derived 'Tip_percent' for tip as a percentage of the total fare. The boxplot below shows the 'Tip_percent' is left skewed. I think it may be a log-normal distribution.

 <img src="9.png">
 
 Code is in Appendix D2.

## Predictive modeling - Feature Engineering

For modeling, I kept time related fields created in Q2. Specifically, they are: 'Lpep_dropoff_weekday', 'lpep_pickup_weekday', 'lpep_pickup_hours', 'Lpep_dropoff_hour'.

I kept distance fields created in Q3 for modeling as well as those location related fields in the original data. Specifically, they are: 'Pickup_To_JFK''Drop_To_JFK', 'Pickup_To_LGA', and 'Drop_To_LGA', 'Is_Pickup_JFK', 'Is_Drop_JFK', 'Is_Pickup_LGA', 'Is_Drop_LGA', 'Is_Airport', 'Pickup_longitude', 'Pickup_latitude', 'Dropoff_longitude', 'Dropoff_latitude'.

I think the 'Tip_percent' may be related to the fare amount without tips. Riders usually see the fare amount first, and then decide the tip amount. So I generated a field named 'Fare_before_tip'.

For categorical fields ('Store_and_fwd_flag', 'Lpep_dropoff_weekday', 'lpep_pickup_weekday', 'lpep_pickup_hours', 'Lpep_dropoff_hour'), I applied mean encoding to generate numeric values with the code below. (Details of mean encoding: https://www.coursera.org/lecture/competitive-data-science/concept-of-mean-encoding-b5Gxv)

Code is in Appendix D3

## Predictive modeling - lightgbm

I applied lightgbm regressor to build a predictive model since lightgbm usually performs well. Instruction of this package is https://lightgbm.readthedocs.io/en/latest/Python-Intro.html

I set parameters as follows:
* "num_leaves" : 30,
* "learning_rate" : 0.01,
* "bagging_fraction" : 0.7,
* "feature_fraction" : 0.7,
* "bagging_frequency" : 5,

I used 4 folds cross validation to evaluate the performance of model via RMSE. The RMSE for 4 folds are: 0.0707, 0.0705, 0.0707, 0.0702. 

lightgbm doesn't return if field has positive or negative impact, but it does return the importance level of fields as follows. 

In [244]:
feat_imp_df.sort_values(by=[('gain','mean')], ascending=False )

,feature,gain,split
,,mean,mean
5,Fare_amount,21.425741,15139.25
2,Dropoff_latitude,9.122492,21724.50
6,Fare_before_tip,7.670212,15087.75
17,Pickup_latitude,6.933870,19160.25
18,Pickup_longitude,6.690003,22537.75
16,Pickup_To_LGA,6.223509,20500.50
3,Dropoff_longitude,6.099072,24151.00
1,Drop_To_LGA,6.086662,22529.00
0,Drop_To_JFK,5.282345,20515.75


The most important ones are 'Fare_amount', 'Dropoff_latitude', 'Fare_before_tip', 'Drop_To_LGA', 'Pickup_latitude', 'Pickup_longitude', which are mainly the location information.

Code is in Appendix D4.

## Predictive modeling - Ridge regrission
I also tried Ridge regression because it can show whether each field has negative or positive impact. Besides, since there are 28 fields and some of them may be not significant, I applied Ridge regression to avoid overfitting.

I used cross validation to evaluate the performance of the model. The performance of four folds on validation sets is 0.0771, 0.0763, 0.0767, 0.0768. The performance is not significantly worse than that of lightgbm which is around 0.070.

Then I used the whole data to build a Ridge regression model and got the coefficients of fields below.

In [245]:
features.sort_values(by=[ 'Coef_ridge' ], ascending=False )

,field,Coef_ridge
27,Lpep_dropoff_hourME,0.443135
26,lpep_pickup_hoursME,0.359518
25,lpep_pickup_weekdayME,0.149339
24,Lpep_dropoff_weekdayME,0.069594
17,Is_Pickup_JFK,0.050468
9,MTA_tax,0.048702
18,Is_Drop_JFK,0.042265
19,Is_Pickup_LGA,0.021814
20,Is_Drop_LGA,0.018302
23,Store_and_fwd_flagME,0.017235


The coefficient result shows that:

* The closer the trip is to airports, the higher the tip percentage. I guess riders taking taxis to airports may have better economy condition since they can afford air flight tickets. 

* The higher the passenger amount, the higher the tip percentage, which makes sense. 

* The longer the time spent, the lower the tip percentage. Long trips always make people tired and upset.

If I have extra time, I'd like to look into the significant level of different fileds, and focus on significant ones. 

Code is in Appendix D5.

## ANOVA - Average speed across five weeks of September

The  plot below shows the mean of 'Average_speed' across five weeks of September. They look quite different. Quantitatively, I applied ANOVA table to test if the mean of 'Average_speed' is the same across weeks of September. 

<img src="11.png">

ANOVA table has some strong assumptions: Normality, Homogeneity of variance and Independent observations. 

* Independent observations can be met since trips are independnet. 

* The  plot below shows the standard deviation of 'Average_speed' across five weeks of September. The standard deviation of Average_speed across different weeks are similar, ranging from 6.4 to 6.7. 

<img src="12.png">

* The plot below is the boxplot of Average_speed for five weeks in September. Based on the plot below, the distribution is left skewed, but most trips are around 20 miles/hour. 

<img src="14.png">

So I think all the asumptions are met. If I have extra time, I'd like to transform 'Average_speed' with exponential function, and also test all of assumptions above strictly and quantitatively. 

The null hypothesis of ANOVA is that all the group means are not significantly different. The result shows that the p value is 0.0, which rejects the null hypothesis, and indicates that Average_speed across five weeks of September are different.

My hypothesis regarding why they differ: I think Week is not the only point of the difference. There may be other related fields, such as 'lpep_pickup_hours', 'Trip_distance', 'Is_Airport' that lead to the difference. 

For example, maybe most of the trips in the first week are not around busy traffic time, so the average speed of that week is higher; or maybe the trips in the first week are not around busy locations such as airports.

Code is in Appendix E2.

## Average speed as a function of time of day

### My Hypothesis

My hypothesis about average trip speed as a function of time of day:

I think the relationship is not linear, but the relationship is significant. Trips around 8:00 am and 18:00 may have lower speed than others because people are getting to work or taking off from work at that time and traffic is always bad.

The following plot shows that there are about two valleys of 'Average_speed' at around 7:00 - 8:00 am and 16:00 - 18:00, which is consistent with my hypothesis. Besides, the relationship is not linear.

<img src="15.png">

### Derive a function between  average speed and time of day

Time of day is too granular to modeling. So I use 'lpep_pickup_hours' to represent the time of day.

I decided not only take 'lpep_pickup_hours' as X to estimate 'Average_speed', but also take other related fields into consideration. Also, I want to keep the interpretability of the function since we may use the result to help avoid heavy traffic. 

As a starting point, I took 'lpep_pickup_hours', 'Week_num', 'Trip_distance' and 'Is_Airport' and build a linear regression model. 

I encoded 'lpep_pickup_hours' and 'Week_num' via mean-encoding because the values of hour or week are not linearly related to 'Average_speed'.

The linear regression result below shows that all the fields are significant: 

* Trips around airports have higher speed which was not as I expected. I think this may be because trips heading to airports avoid busy traffic. 

* The longer the Trip_distance, the lower the speed. 

* The Week_num and lpep_pickup_hours are both significant.

The warning indicate that there are strong multicollinearity or other numerical problems. If I have extra time, I'd like to apply Lasso or Ridge regression.

Code is in Appendix E3.


In [252]:
print(mod2.summary())

                            OLS Regression Results                            
Dep. Variable:          Average_speed   R-squared:                   -4680.409
Model:                            OLS   Adj. R-squared:              -4680.422
Method:                 Least Squares   F-statistic:                -3.679e+05
Date:                Tue, 30 Oct 2018   Prob (F-statistic):               1.00
Time:                        01:57:41   Log-Likelihood:            -1.1070e+07
No. Observations:             1471916   AIC:                         2.214e+07
Df Residuals:                 1471911   BIC:                         2.214e+07
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             459.7172    

# Appendix

## Appendix A

In [ ]:
# Load packages
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import scipy.stats as stats
from math import sin, cos, sqrt, atan2, radians
import lightgbm as lgb
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import Ridge
from sklearn import preprocessing, model_selection, metrics

In [ ]:
# Load data
url = 'https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2015-09.csv'
mdata = pd.read_csv( url )
N, P = mdata.shape 
print( N, P )
mdata.head()

In [ ]:
# Show 21 field names 
mdata.columns.values

In [ ]:
# Calculate number of null values in each field
mdata.isna().sum()

## Appendix B

In [ ]:
# Remove trips with 'Trip_distance' equals 0 because 
mdata_nonZero = mdata.loc[ mdata[ 'Trip_distance' ] != 0 ]

# Calculate the number of trips that have 0 as 'Trip_distance'.
print( 'Percentage of 0 in Trip_distance is', 1 - len( mdata_nonZero ) /N )

# Calculate the mean/median/variance of 'Trip_distance'.
print( 'Mean of Trip_distance is', mdata['Trip_distance' ].mean())
print( 'Median of Trip_distance is', mdata['Trip_distance' ].median())
print( 'Variance of Trip_distance is', mdata['Trip_distance' ].var())
print( 'Mean excluding 0 is', mdata_nonZero['Trip_distance' ].mean())
print( 'Median excluding 0 is', mdata_nonZero['Trip_distance' ].median())
print( 'Variance excluding 0 records is', mdata_nonZero['Trip_distance' ].var())

In [ ]:
# Use distribution plot and box plot to show the distribution of 'Trip_distance'
sns.set(color_codes=True)
fig, (ax1, ax2) = plt.subplots(ncols=2 )
plt1 = sns.distplot( mdata_nonZero[ 'Trip_distance' ] , ax = ax1)
plt1.set(xlabel='Trip_distance(mile)', ylabel='Percentage')
plt1.set_title("Distribution plot of Trip_distance") 
plt2 = sns.boxplot(x = mdata_nonZero[ 'Trip_distance' ], ax = ax2)
plt2.set(xlabel= 'Trip_distance(mile)' )
plt2.set_title("Boxplot of Trip_distance") 
plt.show( )

In [ ]:
# Calculate the number/percentage of trips with 'Trip_distance' greater than 20 miles.
n1 = len(mdata.loc[ mdata[ 'Trip_distance' ] >= 20 ])
print( 'Number of trips with Trip_distance more than 20 miles:', n1 )
print( 'Percentage of trips with Trip_distance more than 20 miles:', \
      n1 / len( mdata_nonZero ) )

In [ ]:
# Remove trips with 'Trip_distance' greater than 20 miles, and build distribution / box plot again
fig, (ax1, ax2) = plt.subplots( ncols=2 )
sns.distplot( (mdata.loc[ mdata[ 'Trip_distance' ] < 20 ])['Trip_distance' ],\
             ax = ax1)
sns.boxplot( (mdata.loc[ mdata[ 'Trip_distance' ] < 20 ])['Trip_distance' ], \
            ax = ax2 )
plt.show()

## Appendix C1

In [ ]:
# Extract hour and weekday information
mdata[ 'lpep_pickup_datetime' ] = pd.to_datetime( \
                                mdata['lpep_pickup_datetime' ], \
                                format='%Y-%m-%d %H:%M:%S')
mdata['lpep_pickup_hours'] = mdata.apply( 
                            lambda x: 
                            x['lpep_pickup_datetime' ].hour, \
                            axis=1)  
mdata['lpep_pickup_weekday'] = mdata.apply(lambda x:
                            x['lpep_pickup_datetime' ].dayofweek,\
                            axis=1)  

mdata[ 'Lpep_dropoff_datetime' ] = pd.to_datetime( mdata['Lpep_dropoff_datetime' ], \
                                                  format='%Y-%m-%d %H:%M:%S')
mdata['Lpep_dropoff_hour'] = mdata.apply(lambda x: 
                                    x['Lpep_dropoff_datetime' ].hour, \
                                    axis=1)
mdata['Lpep_dropoff_weekday'] = mdata.apply(lambda x: 
                                    x['Lpep_dropoff_datetime' ].dayofweek,
                                    axis=1)  

In [ ]:
# Calculate time spent in the trip  
def Time_spent( row ):
    diff = row['Lpep_dropoff_datetime' ] - row['lpep_pickup_datetime' ]
    days = diff.days
    days_to_hours = days * 24
    diff_btw_two_times = (diff.seconds) / 3600
    overall_hours = days_to_hours + diff_btw_two_times
    return( overall_hours )

mdata['Time_spent'] = mdata.apply(lambda row: Time_spent( row )  ,axis=1)  

In [ ]:
# Plot the distribution of 'lpep_pickup_hours', 'Lpep_dropoff_hour'
sns.set(color_codes=True)
fig, (ax1, ax2) = plt.subplots(ncols=2 )
plt1 = sns.distplot( mdata[ 'lpep_pickup_hours' ] , ax = ax1)
plt1.set(xlabel='lpep_pickup_hours', ylabel='Percentage')
plt1.set_title("Distribution of pickup hour of day") 

plt2 = sns.distplot( mdata[ 'Lpep_dropoff_hour' ], ax = ax2)
plt2.set(xlabel= 'Lpep_dropoff_hour' )
plt2.set_title("Distribution of dropoff hour of day") 
plt.show( )

In [ ]:
# Plot the distribution of Time_spent
print( mdata['Time_spent'].mean())
print( mdata['Time_spent'].median())
plt1 = sns.distplot( mdata[ 'Time_spent' ] )
plt1.set(xlabel='Time_spent(hour)', ylabel='Percentage')
plt1.set_title("Distribution of Time_spent") 
plt.show(  )

## Appendix C2

In [ ]:
#  group trip distance by hour of day.
mdata_nonZero = mdata.loc[ mdata[ 'Trip_distance' ] != 0 ]
Distance_by_hour = mdata_nonZero.groupby('lpep_pickup_hours', \
                                as_index = False).agg(\
                                {'Trip_distance':[ 'mean', 'median', 'count', 'var']})

Distance_by_hour

In [ ]:
# Plot trip distance by hour of the day
plt.plot( Distance_by_hour['lpep_pickup_hours'], \
         Distance_by_hour[('Trip_distance', 'mean')]) 

plt.plot( Distance_by_hour['lpep_pickup_hours'], \
         Distance_by_hour[('Trip_distance', 'median')])  
plt.title('Trip distance group by hour of day')
plt.xlabel('hour of day')
plt.ylabel('Trip_distance (mile)')
plt.legend(['Mean', 'Median'], loc='upper right')
plt.show()

In [ ]:
# Plot the number of trips by hour of the day
plt.plot( Distance_by_hour['lpep_pickup_hours'], \
         Distance_by_hour[('Trip_distance', 'count')] )

plt.legend(['Number of Trips'], loc='upper left')
plt.show()

## Appendix C3

In [ ]:
# Box plot of longitude/latitude
fig, (ax1, ax2) = plt.subplots(ncols=2 )
plt1 = sns.boxplot( x = mdata[ 'Pickup_longitude'], ax = ax1 )
plt1.set(xlabel= 'pickup_longitude' )
plt1.set_title("Boxplot of pickup_longitude") 
plt2 = sns.boxplot( x = mdata[ 'Pickup_latitude'], ax = ax2 )
plt2.set(xlabel= 'pickup_latitude' )
plt2.set_title("Boxplot of pickup_latitude") 
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2 )
plt1 = sns.boxplot( x = mdata[ 'Dropoff_longitude'], ax = ax1 )
plt1.set(xlabel= 'Dropoff_longitude' )
plt1.set_title("Boxplot of Dropoff_longitude") 
plt2 = sns.boxplot( x = mdata[ 'Dropoff_latitude'], ax = ax2 )
plt2.set(xlabel= 'Dropoff_latitude' )
plt2.set_title("Boxplot of Dropoff_latitude") 
plt.show()

In [ ]:
# Calculate the distance between pickup/dropoff location and JFK/LGA
JFK = { 'lat': 40.6413, 'lon': -73.7781 }
LGA = { 'lat': 40.7769, 'lon': -73.8740 }
Airports = [ JFK, LGA ]

R = 6373.0 # approximate radius of earth in mile

def Distance_Airport( row ):
    rlt = []
    pick = { 'lat': row[ 'Pickup_latitude'], 'lon': row[ 'Pickup_longitude'] }
    drop = { 'lat': row[ 'Dropoff_latitude'], 'lon': row[ 'Dropoff_longitude'] } 
    locs = { 'pick': pick, 'drop': drop}
    for val in Airports:
        lat1 = radians( val['lat'] )
        lon1 = radians( val['lon'])
        for val2 in [pick, drop]:
            lat2 = radians( val2['lat'] )
            lon2 = radians( val2['lon'] )
            dlon = abs( lon2 - lon1 )
            dlat = abs( lat2 - lat1 )
            a = sin( dlat / 2)**2 + cos( lat1 ) * cos( lat2 ) * sin( dlon / 2)**2
            c = 2 * atan2(sqrt(a), sqrt(1 - a))
            distance = R * c * 0.62
            rlt.append( distance )
    return {'Pickup_To_JFK': rlt[0], 'Drop_To_JFK': rlt[1],\
            'Pickup_To_LGA': rlt[2], 'Drop_To_LGA': rlt[3] }

mdata = mdata.merge( mdata[['Pickup_latitude','Dropoff_latitude', \
                        'Pickup_longitude', 'Dropoff_longitude']].apply(lambda \
                        row: pd.Series( Distance_Airport(row) ), axis=1),\
             left_index=True, right_index=True)

In [ ]:
buffer = 1 # mile
def Label_Airport( row ):
    rlt = []
    for key in ['Pickup_To_JFK','Drop_To_JFK','Pickup_To_LGA','Drop_To_LGA']:
        if row[key] <= buffer:
            rlt.append( 1 )
        else:
            rlt.append( 0 )
    return {'Is_Pickup_JFK': rlt[0], 'Is_Drop_JFK': rlt[1],\
            'Is_Pickup_LGA': rlt[2], 'Is_Drop_LGA': rlt[3] }

mdata = mdata.merge( mdata[['Pickup_To_JFK','Drop_To_JFK',\
                            'Pickup_To_LGA','Drop_To_LGA']].apply(lambda \
                            row: pd.Series( Label_Airport( row ) ), axis=1),\
             left_index=True, right_index=True)

mdata['Is_Airport'] = mdata.apply( lambda row: row['Is_Pickup_JFK'] == 1 \
                                    or row['Is_Drop_JFK'] == 1 or row['Is_Pickup_LGA'] == 1 
                                        or row['Is_Drop_LGA']==1, axis = 1)

In [ ]:
# Calculate the number of trips that originated or terminated at any of airports
mdata_nonZero_Loc = mdata.loc[ (mdata['Pickup_longitude']!=0) &\
                              (mdata['Pickup_latitude']!=0) & \
                              (mdata['Dropoff_longitude']!=0) &\
                              (mdata['Dropoff_latitude']!=0)]

Is_airport = mdata_nonZero_Loc.loc[ mdata_nonZero_Loc['Is_Airport'] == 1 ]

print( 'number of trips that originate or terminate \
at one of the NYC area airports: ', len(Is_airport) )

print( 'percentage of trips that originate or terminate \
at one of the NYC area airports: ', len(Is_airport)/N )

In [ ]:
# the average fare, and any other interesting characteristics of trips originate or terminate at airports
Airport_trips = mdata_nonZero_Loc.groupby(['Is_Airport']).agg( { 'Fare_amount':[ 'count', 'mean'],
                                                 'Trip_distance': [ 'mean', 'var' ], 
                                                 'lpep_pickup_hours': ['mean', 'var'],
                                                 'Time_spent': ['mean', 'var']} )

In [ ]:
# May add some location plots

## Appendix D1

In [ ]:
# Box plot of Tip amount across different payment types
mapping = { 1: 'Credit_card', 2: 'Cash', 3: 'No_charge', \
           4: 'Dispute', 5: 'Unknown', 6: 'Voided'}

mdata['Payment_type_1'] = mdata['Payment_type'].apply( lambda x: mapping[ x ] )

plt1 = sns.boxplot(x="Tip_amount", y="Payment_type_1", data= mdata )
plt1.set_title("Boxplot of Tip_amount across five payment types") 
plt1 

In [ ]:
# Box plot of Total amount across different payment types
plt2 = sns.boxplot(x="Total_amount", y="Payment_type_1", data= mdata )
plt2.set_title("Boxplot of Total_amount across five payment types") 
plt2 

## Appendix D2

In [ ]:
# Keep trips with payment type equals credit card
mdata_modeling = mdata.loc[ mdata[ 'Payment_type_1'] == 'Credit_card']
print( mdata_modeling.shape )

In [ ]:
# Calculate the number of records with non-positive Total amount.
temp = mdata_modeling.loc[ mdata_modeling[ 'Total_amount' ] <= 0 ] 
print( temp.shape )
temp[['Tip_amount', 'Total_amount', 'Trip_distance']].head()

# Remove records with Total_amount <= 0
mdata_modeling = mdata_modeling.loc[ mdata_modeling[ 'Total_amount'] > 0 ]
print( mdata_modeling.shape)

In [ ]:
# Creat 'Tip_percent' for tip as a percentage of the total fare.
mdata_modeling['Tip_percent'] = mdata_modeling['Tip_amount']/\
                mdata_modeling['Total_amount']

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2 )
plt1 = sns.distplot( mdata_modeling[ 'Tip_percent' ], ax = ax1)
plt1.set(xlabel= 'Tip_percent' )
plt1.set_title("Distribution of Tip_percent") 

plt2 = sns.boxplot(x =mdata_modeling[ 'Tip_percent' ], ax = ax2)
plt2.set(xlabel= 'Tip_percent' )
plt2.set_title("Boxplot of Tip_percent") 
plt.show()

 ## Appendix D3

In [ ]:
# Generate 'Fare_before_tip'
mdata_modeling['Fare_before_tip'] = mdata_modeling['Fare_amount'] + \
        mdata_modeling['Extra'] + mdata_modeling[ 'MTA_tax' ] +\
        mdata_modeling['improvement_surcharge'] + mdata_modeling['Tolls_amount'] 

In [ ]:
# Given the analysis in Q3, I removed records with 0 Pickup_longitude/Pickup_latitude
mdata_modeling = mdata_modeling.loc[ ( mdata_modeling[ 'Pickup_longitude'] != 0 ) \
                                    & (mdata_modeling['Pickup_latitude' ] != 0 ) ]

In [ ]:
# Mean encoding for categorical fields
cate_cols = [ 'Store_and_fwd_flag', 'Lpep_dropoff_weekday', \
             'lpep_pickup_weekday', 'lpep_pickup_hours', 'Lpep_dropoff_hour']

for col in cate_cols:
    agg = mdata_modeling.groupby([ col ], as_index = False )['Tip_percent'].mean()
    agg = agg.rename( index=str, columns= {'Tip_percent': col+'ME' })
    mdata_modeling = mdata_modeling.merge( agg, how = 'left', on = col)

## Appendix D4

In [ ]:
# The X variables for regression
X_cols = [ 'RateCodeID', 'Pickup_longitude',
       'Pickup_latitude', 'Dropoff_longitude', 'Dropoff_latitude',
       'Passenger_count', 'Trip_distance', 'Fare_amount', 'Extra',
       'MTA_tax',  'Tolls_amount', 'improvement_surcharge',
       'Trip_type ', 'Pickup_To_JFK', 'Drop_To_JFK', 'Pickup_To_LGA', 'Drop_To_LGA',
       'Is_Pickup_JFK', 'Is_Drop_JFK', 'Is_Pickup_LGA', 'Is_Drop_LGA', 
       'Time_spent', 'Fare_before_tip','Store_and_fwd_flagME', 'Lpep_dropoff_weekdayME',
       'lpep_pickup_weekdayME', 'lpep_pickup_hoursME',
       'Lpep_dropoff_hourME']

In [ ]:
def run_lgb(train_X, train_y, val_X, val_y ):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 30,
        "learning_rate" : 0.01,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.7,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 10000, valid_sets=[lgval],\
                      early_stopping_rounds= 500, verbose_eval=5000,\
                      evals_result=evals_result)
    
    gain = model.feature_importance('gain')
    fold_importance_df = pd.DataFrame({'feature':model.feature_name(), 
                   'split': model.feature_importance('split'), 
                   'gain':100 * gain / gain.sum()}).sort_values('gain',\
                                                    ascending=False)
    return model, evals_result, fold_importance_df

In [ ]:
# 4 folds cross validation
def lightgbm_regressor( trainx, trainy ):
    kf = model_selection.KFold(n_splits= 4, shuffle=True, random_state=2017)
    pred_test_full = 0
    feature_importance_df = pd.DataFrame()
    for dev_index, val_index in kf.split(trainx):
        dev_X, val_X = trainx.loc[dev_index,:], trainx.loc[val_index,:]
        dev_y, val_y = trainy[dev_index], trainy[val_index]
        model, evals_result, fold_imp_df = run_lgb(dev_X, dev_y, val_X, val_y )
        feature_importance_df = pd.concat([feature_importance_df,fold_imp_df],axis=0)
    return( feature_importance_df )  

In [ ]:
# Apply lgboost
train_x = mdata_modeling[ X_cols ]
train_y = mdata_modeling['Tip_percent']
feat_impt_df = lightgbm_regressor( train_x, train_y  )

In [ ]:
# Show Top 5 features with highest importance level
feat_imp_df = feat_impt_df.groupby(['feature'], \
                                   as_index = False ).agg( {'gain': ['mean'],
                                                          'split': ['mean']
                                                         })
feat_imp_df.sort_values(by=[('gain','mean')], ascending=False ).head()

## Appendix D5

In [ ]:
# remove null values of the data.
mdata_modeling2 = mdata_modeling.drop( ['Ehail_fee'], axis = 1 )
mdata_modeling2 = mdata_modeling2.dropna()
train_x2 = mdata_modeling2[X_cols]
train_y2 = mdata_modeling2['Tip_percent']

In [ ]:
def cv_ridge( mdata ):
    kf = model_selection.KFold(n_splits= 4, shuffle=True, random_state=2017)
    rlt = []
    for dev_index, val_index in kf.split( mdata ):
        dev, val = mdata.loc[dev_index,:], mdata.loc[val_index,:]
        dev, val = dev.dropna(), val.dropna()
        dev_X, dev_y = dev[X_cols], dev['Tip_percent']
        val_X, val_y = val[X_cols], val['Tip_percent']
        clf = Ridge(alpha= 0.5 )
        clf.fit( dev_X, dev_y ) 
        val_pred = clf.predict( val_X )
        rlt.append( sqrt( (( clf.predict( val_X )- val_y )**2).mean() ) )
    return rlt



In [ ]:
cv_perf = cv_ridge( mdata_modeling2)

In [ ]:
cv_perf

In [ ]:
clf = Ridge(alpha= 0.5 )
clf.fit( train_x2, train_y2 )

In [ ]:
# Calculate the coefficients of fields
features = pd.DataFrame( train_x2.columns.values , columns = ['field'])
features['Coef_ridge'] = clf.coef_
features.sort_values(by=[ 'Coef_ridge' ], ascending=False )

# Q5: Option A: Distributions of speed

Analyzing the distribution of speed can help identify when the traffic gets base, and help riders and drivers save time.

For this analysis, I removed trips with non-positive 'Trip_distance' or non-positive 'Time_spent' because those don't make sense. There are 1.5 million trips in total.

I derived a 'Average_speed' field representing the average speed over the course of a trip, and a 'Week_num' feature indicating in which week of September that trip happended.

The boxplot below shows that some 'Average_speed' are quite large, which doesn't make sense. So I capped the 'Average_speed' with 200 mile/hour, and removed trips with 'Average_speed' greater than 200 mile/hour

10.png

 Code is in Appendix E1

## Appendix E1

In [ ]:
# removed trips with non-positive 'Trip_distance' or non-positive 'Time_spent'
mdata_speed = mdata.loc[ (mdata[ 'Trip_distance' ] > 0 ) &\
                        (mdata[ 'Time_spent' ] > 0 )]

print( mdata_speed.shape )

In [ ]:
# Calculate 'Average_speed' as follows:
mdata_speed[ 'Average_speed' ] = mdata_speed.apply( lambda \
                    row: row['Trip_distance'] / row['Time_spent'], axis = 1 )

In [ ]:
# Derive 'Week_num' indicating which week of September that trip happens
mdata_speed[ 'Week_num' ] = mdata_speed.apply( lambda \
                    row: row['lpep_pickup_datetime'].isocalendar()[1], axis = 1 )

week_mapping = { 36: '1st', 37: '2nd', 38: '3rd',39: '4th', 40: '5th'}
mdata_speed['Week_num_1'] = mdata_speed['Week_num'].apply( lambda\
                                        x: week_mapping[ x ] )

In [ ]:
# Boxplot of Average_speed across different weeks of September
plt1 = sns.boxplot(x="Average_speed", y="Week_num_1", data= mdata_speed )
plt1.set_title("Boxplot of Average_speed across different weeks of September") 
plt1

In [ ]:
# removed trips with 'Average_speed' greater than 200 mile/hour
mdata_speed2 = mdata_speed.loc[ mdata_speed['Average_speed'] <= 200] 
print( mdata_speed2.shape )

## Appendix E2

In [ ]:
# plot of Mean/Standard Deviation of 'Average_speed' across five weeks in September
Week_mean = mdata_speed2.groupby( ['Week_num'],\
                as_index = False ).agg( {'Average_speed': ['mean','std'] })

plt.scatter(x = Week_mean[( 'Week_num', )], \
            y = Week_mean[ ( 'Average_speed', 'mean') ])

plt.title('Mean of Average_speed across five weeks in September')
plt.xlabel('Week in September')
plt.ylabel('Average_speed(mile/hour)')
plt.show()

plt.scatter(x = Week_mean[( 'Week_num', )], \
            y = Week_mean[ ( 'Average_speed', 'std') ])
plt.title('Standard Deviation of Average_speed across five weeks in September')
plt.xlabel('Week in September')
plt.ylabel('Average_speed(mile/hour)')

plt.show()

plt3 = sns.boxplot(x="Average_speed", y="Week_num_1", data= mdata_speed2 )
plt3.set(xlabel= 'Average_speed' )
plt3.set_title("Boxplot of Average speed across five weeks of September ") 


In [ ]:
# ANOVA Test
ANOVA_rlt = stats.f_oneway( mdata_speed2[ 'Average_speed' ][ mdata_speed2['Week_num_1'] == '1st'], 
             mdata_speed2[ 'Average_speed' ][ mdata_speed2['Week_num_1'] == '2nd'],
             mdata_speed2[ 'Average_speed' ][ mdata_speed2['Week_num_1'] == '3rd'],
             mdata_speed2[ 'Average_speed' ][ mdata_speed2['Week_num_1'] == '4th'],
             mdata_speed2[ 'Average_speed' ][ mdata_speed2['Week_num_1'] == '5th'])

ANOVA_rlt

## Appendix E3

In [ ]:
Hour_mean = mdata_speed2.groupby([ 'lpep_pickup_hours' ], \
                                 as_index = False )['Average_speed'].mean()

plt.scatter(x = Hour_mean[( 'lpep_pickup_hours')], \
            y = Hour_mean[ ( 'Average_speed') ])

plt.title('Scatter plot of Averate_speed over hour of day')
plt.xlabel('hour of day')
plt.ylabel('Average_speed(mile/hour)')

plt.show()

In [ ]:
# mean-encode 'lpep_pickup_hours' and 'Week_num'  
cate_cols_speed = [ 'lpep_pickup_hours', 'Week_num' ]

for col in cate_cols_speed:
    agg = mdata_speed2.groupby([ col ], as_index = False )['Average_speed'].mean()
    agg = agg.rename( index=str, columns= {'Average_speed': col+'ME' })
    mdata_speed2 = mdata_speed2.merge( agg, how = 'left', on = col)

In [ ]:
mod = smf.ols(formula='Average_speed ~ lpep_pickup_hoursME + Week_numME \
                + Trip_distance + Is_Airport' , data=mdata_speed2)
mod2 = mod.fit()
print(mod2.summary())